In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re as re
import requests
from IPython.display import display

display(np.__version__)
display(pd.__version__)

'1.16.5'

'0.25.1'

In [2]:
homicide = pd.read_csv("database.csv",dtype={'Perpetrator Age':str})
# after running the above without the dtype specifier there was an error.
# The problem is that record with 'Record ID' = 634667 has a single blank space for "Perpetrator Age" and that one entry
# messes up the read_csv algorithm which tries to guess the type of data in each column.  It assumes everything is integer
# but then doesn't need to know what to do with the " " entry.  This is an unfortunate limitation!  Is there a way around it?
# What I've done is forced the column to be interpreted as a string, then convert it to a int below, after substituting the
# space with an emptry string
homicide['Perpetrator Age'] = pd.to_numeric(homicide['Perpetrator Age'].str.replace(' ',''))

In [3]:
homicide.head()

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15.0,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0.0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


- consider changing column names to remove spaces
- may want to remove some columns as there are a lot!

In [4]:
homicide.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638454 entries, 0 to 638453
Data columns (total 24 columns):
Record ID                638454 non-null int64
Agency Code              638454 non-null object
Agency Name              638454 non-null object
Agency Type              638454 non-null object
City                     638454 non-null object
State                    638454 non-null object
Year                     638454 non-null int64
Month                    638454 non-null object
Incident                 638454 non-null int64
Crime Type               638454 non-null object
Crime Solved             638454 non-null object
Victim Sex               638454 non-null object
Victim Age               638454 non-null int64
Victim Race              638454 non-null object
Victim Ethnicity         638454 non-null object
Perpetrator Sex          638454 non-null object
Perpetrator Age          638453 non-null float64
Perpetrator Race         638454 non-null object
Perpetrator Ethnicity    638

In [5]:
homicide.describe()

,Record ID,Year,Incident,Victim Age,Perpetrator Age,Victim Count,Perpetrator Count
count,638454.00000,638454.000000,638454.000000,638454.000000,638453.000000,638454.000000,638454.000000
mean,319227.50000,1995.801102,22.967924,35.033512,20.322697,0.123334,0.185224
std,184305.93872,9.927693,92.149821,41.628306,17.886842,0.537733,0.585496
min,1.00000,1980.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,159614.25000,1987.000000,1.000000,22.000000,0.000000,0.000000,0.000000
50%,319227.50000,1995.000000,2.000000,30.000000,21.000000,0.000000,0.000000
75%,478840.75000,2004.000000,10.000000,42.000000,31.000000,0.000000,0.000000
max,638454.00000,2014.000000,999.000000,998.000000,99.000000,10.000000,10.000000


In [6]:
#homicide[(homicide['City'] == 'Tulsa') & (homicide['Year'] == 2014)]
display(homicide.nunique())
homicide['Crime Type'].value_counts()

Record ID                638454
Agency Code               12003
Agency Name                9216
Agency Type                   7
City                       1782
State                        51
Year                         35
Month                        12
Incident                   1000
Crime Type                    2
Crime Solved                  2
Victim Sex                    3
Victim Age                  101
Victim Race                   5
Victim Ethnicity              3
Perpetrator Sex               3
Perpetrator Age             100
Perpetrator Race              5
Perpetrator Ethnicity         3
Relationship                 28
Weapon                       16
Victim Count                 11
Perpetrator Count            11
Record Source                 2
dtype: int64

Murder or Manslaughter        629338
Manslaughter by Negligence      9116
Name: Crime Type, dtype: int64

In [7]:
homicide[homicide['Perpetrator Race'] == homicide['Victim Race']]

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15.0,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42.0,White,Unknown,Acquaintance,Strangulation,0,0,FBI
5,6,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,May,1,Murder or Manslaughter,...,Unknown,Male,36.0,White,Unknown,Acquaintance,Rifle,0,0,FBI
7,8,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,June,1,Murder or Manslaughter,...,Unknown,Male,35.0,White,Unknown,Wife,Knife,0,0,FBI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638446,638447,WY01300,Natrona County,Sheriff,Natrona,Wyoming,2014,September,1,Murder or Manslaughter,...,Not Hispanic,Male,48.0,White,Not Hispanic,Mother,Handgun,0,0,FBI
638450,638451,WY01700,Sheridan County,Sheriff,Sheridan,Wyoming,2014,June,1,Murder or Manslaughter,...,Unknown,Male,57.0,White,Unknown,Acquaintance,Handgun,0,0,FBI
638451,638452,WY01701,Sheridan,Municipal Police,Sheridan,Wyoming,2014,September,1,Murder or Manslaughter,...,Unknown,Female,22.0,Asian/Pacific Islander,Unknown,Daughter,Suffocation,0,0,FBI
638452,638453,WY01800,Sublette County,Sheriff,Sublette,Wyoming,2014,December,1,Murder or Manslaughter,...,Not Hispanic,Male,31.0,White,Not Hispanic,Stranger,Knife,0,1,FBI


In [8]:
homicide['Victim Race'].value_counts().sort_values(ascending=False)
#temp.value_counts()

White                            317422
Black                            299899
Asian/Pacific Islander             9890
Unknown                            6676
Native American/Alaska Native      4567
Name: Victim Race, dtype: int64

In [9]:
homicide['Perpetrator Race'].value_counts().sort_values(ascending=False)

White                            218243
Black                            214516
Unknown                          196047
Asian/Pacific Islander             6046
Native American/Alaska Native      3602
Name: Perpetrator Race, dtype: int64

In [10]:
homicide.groupby('Year')['Victim Age'].aggregate(['count','mean'])

,count,mean
Year,,
1980,23092,35.351637
1981,21208,36.056300
1982,20544,35.670415
1983,19653,35.857681
1984,18093,37.139944
1985,18386,36.221962
1986,20125,35.983453
1987,18783,35.340201
1988,18779,34.704244


In [11]:
homicide.groupby('Year')['Perpetrator Age'].aggregate(['count','mean'])

,count,mean
Year,,
1980,23092,21.835571
1981,21208,22.973595
1982,20544,22.525701
1983,19653,22.462016
1984,18093,22.422760
1985,18386,22.670021
1986,20125,22.299528
1987,18783,21.689081
1988,18779,21.086320


***Some more gun violence datasets to answer exploratory questions***

In [19]:
df_gunviolence1 = pd.read_excel("Gun-related deaths by age and sex 1999-2018.xlsx")
df_gunviolence1.head()

,Year,Sex,Type,Unnamed: 3,Number,< 1 year,1-4 years,5-14 years,15-24 years,25-34 years,35-44 years,45-54 years,55-64 years,65-74 years,75-84 years,85+ years,Not Stated
0,2018,Both sexes,Total - all intents,NaN,39740.0,7,91,437.0,7411.0,8100.0,6027.0,5323.0,5353.0,3662.0,2365.0,963.0,1.0
1,2018,Both sexes,Suicide,NaN,24432.0,0,0,203.0,2995.0,3429.0,3222.0,3787.0,4421.0,3237.0,2215.0,923.0,0.0
2,2018,Both sexes,Assault,NaN,13958.0,6,54,191.0,4107.0,4348.0,2569.0,1382.0,802.0,347.0,114.0,37.0,1.0
3,2018,Both sexes,Legal intervention,NaN,539.0,0,0,0.0,87.0,160.0,148.0,67.0,55.0,18.0,4.0,0.0,0.0
4,2018,Both sexes,Preventable/accidental,NaN,458.0,0,30,24.0,129.0,79.0,46.0,45.0,51.0,34.0,19.0,1.0,0.0


In [12]:
df_gunviolence2 = pd.read_csv("Mother Jones' Investigation- US Mass Shootings, 1982-2017 - US mass shootings.csv")

In [13]:
df_gunviolence2.head()

,Case,Location,Date,Year,Summary,Fatalities,Injured,Total victims,Venue,Prior signs of mental health issues,...,Where obtained,Type of weapons,Weapon details,Race,Gender,Sources,Mental Health Sources,latitude,longitude,Type
0,Texas First Baptist Church massacre,"Sutherland Springs, TX",11/5/17,2017,"Devin Patrick Kelley, a 26-year-old ex-US Air ...",26,20,46+,Religious,Yes,...,Purchased in April 2016 from an Academy Sports...,semiautomatic rifle,Ruger AR-556; Kelley also possessed semiautoma...,White,M,https://www.washingtonpost.com/news/morning-mi...,http://www.expressnews.com/news/local/article/...,32.780105,-96.800008,Mass
1,Walmart shooting in suburban Denver,"Thornton, CO",11/1/17,2017,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,Other,Unclear,...,NaN,NaN,NaN,White,M,https://www.nytimes.com/2017/11/01/us/thornton...,NaN,43.060567,-88.106479,Mass
2,Edgewood businees park shooting,"Edgewood, MD",10/18/17,2017,"Radee Labeeb Prince, 37, fatally shot three pe...",3,3,6,Workplace,Unclear,...,Unclear,handgun,.380-caliber; make unclear,Black,M,http://www.baltimoresun.com/news/maryland/harf...,NaN,NaN,NaN,Mass
3,Las Vegas Strip massacre,"Las Vegas, NV",10/1/17,2017,"Stephen Craig Paddock, 64, rained a barrage of...",58+,489+,547+,Other,TBD,...,Two gun shops in Nevada,"23 firearms, mostly rifles; including scopes, ...","AR-15-style and AK-47-style rifles and ""a larg...",White,M,https://www.nytimes.com/2017/10/02/us/stephen-...,https://www.nytimes.com/2017/10/13/us/stephen-...,32.693397,-97.470671,Mass
4,San Francisco UPS shooting,"San Francisco, CA",6/14/17,2017,"Jimmy Lam, 38, fatally shot three coworkers an...",3,2,5,Workplace,Yes,...,Unclear; the firearm was stolen in Utah. A sec...,two handguns,MAC-10-style “assault pistol”; 30-round magazi...,Asian,M,http://www.nbcbayarea.com/news/local/Active-Sh...,http://www.ktvu.com/news/ktvu-local-news/26177...,NaN,NaN,Mass


***Second dataset - STATE WISE GUN LAWS 1991 - 2020***

This data frame consists of total number of gun laws in each state from 1991 to 2020. This dataset will help with analyzing gun violence trends with 'number of gun laws' as one of the factors

Fields - 

- state : State name 
- year : Year
- lawtotal : Total number of gunlaws in effect that year

In [17]:
gunlawscore = pd.read_excel("DATABASE_0.xlsx")
gunlawscore_df = gunlawscore[['state','year','lawtotal']].copy()
gunlawscore_df.head()

,state,year,lawtotal
0,Alabama,1991,15
1,Alabama,1992,16
2,Alabama,1993,16
3,Alabama,1994,16
4,Alabama,1995,16


In [18]:
gunlawscore_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 3 columns):
state       1500 non-null object
year        1500 non-null int64
lawtotal    1500 non-null int64
dtypes: int64(2), object(1)
memory usage: 35.3+ KB


***Third dataset - State-Level Estimates of Household Firearm Ownership***

In [ ]:
# Loading state wise household firearm ownership data into dataframe
df_firearm_ownership = pd.read_csv("TL-354-State-Level Estimates of Household Firearm Ownership.csv")

In [ ]:
#Extracting fields of interest to a new dataframe
df_firearm_ownership_data = df_firearm_ownership[['Year','STATE','HFR','universl','permit']].copy()

This data frame consists of estimates of  the proportion of adult, noninstitutionalized residents in each U.S. state and year (1980–2016) who live in a household with a firearm
- Year : Year
- STATE : State name 
- HFR : Factor scores for household firearm ownership latent factor
- universl : State has universal background checks law (1=yes; 0=no)
- permit : State has permit to purchase law (1=yes; 0=no)


In [ ]:
df_firearm_ownership_data.head()

***Fourth dataset - State-wise US Population***

In [ ]:
# Units:Thousands of Persons
df_population = pd.read_csv("States_Population_Data.csv")
df_population.head()

In [ ]:
df1_transposed = df_population.transpose()
df1_transposed

In [ ]:
df_population.head()